In [15]:
import numpy as np
import pandas as pd
import regex as re
from multiprocessing.pool import ThreadPool
import random
from scraper import StockInfo
from dcfModel import DCF

In [2]:
tick = StockInfo("NVDA")

In [13]:
balancesheet = tick.get_BalanceSheet()
balancesheet[balancesheet["Breakdown"] == "Total Debt"].iloc[:,1:2].astype(float).values[0][0]

11855.0

In [16]:
df = pd.read_csv('csv/tickers.csv')
df

,ticker,industry,operatingMarginStart,operatingMarginTarget,operatingMarginDistribution,operatingMarginGrowthTrend,RevenueGrowthRateStart,RevenueGrowthRateEnd,RevenueGrowthRateTerminalValue,RevenueGrowthRateDistribution,RevenueGrowthRateGrowthTrend,CostofcapitalStart,CostofcapitalEnd,CostofcapitalTrend,CostofcapitalDistribution,salesToCapital,salesToCapitalDistribution,TaxRateStart,TaxRateEnd,TaxRateEndDistribution
0,NVDA,Semiconductor,20,40,normal,linear,34,10,6,uniform,cyclical,12.21,8.4,linear,triangular,1.15,normal,10,20,uniform
